In [3]:
# Criando classe Arvore

def find_leaves(node, l):
    for filho in node.filhos:
        if filho.gi == 0:
            l.append(filho.df)
            return
        else: find_leaves(filho, l)
                
def format_leaves(df, target):
    ret = []
    aux = df.columns.tolist()
    aux.remove(target)
    p = df[target].value_counts(1)
    for col in aux: ret.append([col, df[col].unique().tolist()])
    if len(p) == 2:  ret.append(['s', p.loc[p.index == 1].iloc[0]])
    elif len(p) == 1 and p.index.unique()[0] == 0: ret.append(['s', 0])
    elif len(p) == 1 and p.index.unique()[0] == 1: ret.append(['s', 1])
    return ret

def format_new_instance(new):
    new_d = {}
    for index in new.index:
        new_d[index] = new[index]
    return new_d

class Arvore:

    def __init__(self, 
                 df, var_resposta, regras = None,
                 atributo_origem = '-',
                 atributo_origem_valor = '-', 
                 profundidade = 0):

        self.df = df
        self.var_resposta = var_resposta
        self.regras = regras
        self.filhos = []
        self.atributo_melhor = '-'
        self.atributo_origem = atributo_origem
        self.atributo_origem_valor = atributo_origem_valor
        self.gi = 0
        self.profundidade = profundidade
        
        
    def __repr__(self):
        return f'''Nó {self.profundidade} da Árvore de Decisão.\nCaracterísticas do No:
            Dataframe de {self.df.shape[0]} linhas.
            A origem desse nó é o atributo {self.atributo_origem} com valor {self.atributo_origem_valor}
            Melhor atributo desse nó é {self.atributo_melhor} com GI de {self.gi}'''

    @staticmethod
    def entropia_binaria(p):

        if p == 0 or p == 1:
            return 0

        n = 1 - p
        return -(p*np.log2(p) + n*np.log2(n))

    @staticmethod
    def ganho_informacao(df, coluna, var_resposta):

        seq_poss = df[coluna].unique()

        ent_inicial = Arvore.entropia_binaria(df[var_resposta].value_counts(1).iloc[0])

        conj = [df[df[coluna] == i] for i in seq_poss]
        ent = [Arvore.entropia_binaria(i[var_resposta].value_counts(1).iloc[0]) for i in conj]
        n_elem = [i.shape[0] for i in conj]

        total = sum(n_elem)

        ent_atr = 0
        for i in range(len(conj)):
            ent_atr += ent[i]*n_elem[i]/total

        return ent_inicial - ent_atr

    def melhor_atributo(self):
        '''
        Esse método acha o melhor atributo em relação ao ganho de informação.
        '''

        colunas_a_testar = self.df.columns.to_list()
        colunas_a_testar.remove(self.var_resposta)

        maior_v = 0
        maior_k = '-'

        for col in colunas_a_testar:

            v = Arvore.ganho_informacao(self.df, col, self.var_resposta)
            if v > maior_v:
                maior_v = v
                maior_k = col

        return maior_k, maior_v

    def realiza_quebra(self):
        '''
        Quebra o conjunto de dados inicial em N subconjuntos de acordo com 
        o melhor atributo.
        '''

        self.atributo_melhor, self.gi = self.melhor_atributo()

        if self.atributo_melhor == '-':
            return 

        possibilidades = self.df[self.atributo_melhor].unique()

        for p in possibilidades:
            mascara = self.df[self.atributo_melhor] == p
            sub_conj = self.df.loc[mascara, :]
            self.filhos.append(Arvore(sub_conj, 
                                      self.var_resposta, 
                                      atributo_origem = self.atributo_melhor,
                                      atributo_origem_valor = p,
                                      profundidade = self.profundidade + 1))


    def cria_galhos(self):

        self.realiza_quebra()

        for i in self.filhos:

            p0 = i.df[i.var_resposta].value_counts(1).iloc[0]
            ent = Arvore.entropia_binaria(p0)

            if ent > 0:
                i.cria_galhos()

    def cria_regras(self):
        leaves = []                   
        find_leaves(self, leaves) 
    
        leaf_as_dict_list = []
        for data in leaves:
            leaf_as_dict_list.append(format_leaves(data, self.var_resposta)) 
                                                           
        rules_dictionary = {}                           
        for n in range(0, len(leaf_as_dict_list)):   
                                                
            rules_dictionary[n] = {}
            for i in range(0, len(leaf_as_dict_list[n]) - 1):
                rules_dictionary[n][leaf_as_dict_list[n] [i] [0]] = leaf_as_dict_list[n] [i][1] [0]
                rules_dictionary[n][leaf_as_dict_list[n] [4] [0]] = leaf_as_dict_list[n] [4] [1]
            
        self.regras = rules_dictionary
    
    def predicao(self, new_instance): 
        hit_count_by_index = []
        for n in range(0, len(self.regras)):
            hit_count = 0
            for key in new_instance.keys():
                if new_instance[key] == self.regras[n][key]:
                    hit_count += 1
            hit_count_by_index.append([hit_count, n]) 
        hit_count_by_index.sort()
        
        return ('prob.:', self.regras[hit_count_by_index[-1][1]]['s'], 'index:', hit_count_by_index[-1][1], 
             'hit count:', hit_count_by_index[-1][0])
    
# o método 'predicao' não aceitará que o dado recebido contenha a variável alvo

# saída do método 'predicao':
# 'prob.:' <- probabilidade de sobrevivência da nova instância
# 'index:' <- em qual dicionário do dicionário de regras, a nova instância se encaixa melhor
# 'hit count:' <- número de atributos da nova instância que são compatíveis com as características da folha

Nó 0 da Árvore de Decisão.
Características do No:
            Dataframe de 15 linhas.
            A origem desse nó é o atributo - com valor -
            Melhor atributo desse nó é - com GI de 0


('prob.:', 1, 'index:', 0, 'hit count:', 4)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('titanic.csv')
df_amostra = df.sample(15, random_state = 1)
df_amostra = df_amostra.drop(columns = ['PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'Embarked'])
df_amostra['Sex'] = df_amostra['Sex'].str.replace('female', '0').str.replace('male', '1').astype(int)

arv = Arvore(df_amostra, 'Survived')
print(arv)

arv.cria_galhos()
arv.cria_regras()

df_targetless = df_amostra.drop(columns = ['Survived']) # não aceita que dado contenha variável alvo
arv.predicao(df_targetless.iloc[0, :])

#  arv.regras <- para visualizar dicinário criado em 'cria_regras'